In [2]:
import numpy as np
import sys

from goatools.obo_parser import GODag
from goatools.base import get_godag
import pandas as pd
import requests, sys
import numpy as np
import time
#go = get_godag("go-basic.obo", optional_attrs={'relationship'})
import multiprocessing
from itertools import repeat

import time

from Similarity import Similarity_of_Two_GOTerms, Similarity_of_Set_of_GOTerms
from untitled import f

import sys
import pickle
from tqdm.notebook import trange, tqdm

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2024-06-17) 51,502 Terms; optional_attrs(relationship)
  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2024-06-17) 51,502 Terms; optional_attrs(relationship)


In [3]:
affymetrix_GO_terms_dict = dict(zip(pd.read_pickle('Data/affymetrix_GO_terms_dict.pkl')['ID'],pd.read_pickle('Data/affymetrix_GO_terms_dict.pkl')['GO term']))
data = pd.read_pickle('Data/data_normalized.pkl')
colunas = list(data.columns)
colunas.remove('samples')
colunas.remove('type')

In [4]:
affymetrix_GO_terms_dict

{'1007_s_at': ['GO:0001558',
  'GO:0001952',
  'GO:0006468',
  'GO:0007155',
  'GO:0007169',
  'GO:0007565',
  'GO:0007566',
  'GO:0007595',
  'GO:0008285',
  'GO:0010715',
  'GO:0014909',
  'GO:0016310',
  'GO:0018108',
  'GO:0030198',
  'GO:0038063',
  'GO:0038063',
  'GO:0038083',
  'GO:0043583',
  'GO:0044319',
  'GO:0046777',
  'GO:0060444',
  'GO:0060749',
  'GO:0061302'],
 '1053_at': ['GO:0000278',
  'GO:0000722',
  'GO:0000723',
  'GO:0006260',
  'GO:0006271',
  'GO:0006281',
  'GO:0006283',
  'GO:0006289',
  'GO:0006297',
  'GO:0015979',
  'GO:0015995',
  'GO:0032201'],
 '117_at': ['GO:0000902',
  'GO:0006200',
  'GO:0006950',
  'GO:0006986',
  'GO:0034605',
  'GO:0042026',
  'GO:0070370'],
 '121_at': ['GO:0001655',
  'GO:0001656',
  'GO:0001658',
  'GO:0001822',
  'GO:0001823',
  'GO:0003337',
  'GO:0006351',
  'GO:0006355',
  'GO:0007275',
  'GO:0007417',
  'GO:0009653',
  'GO:0030154',
  'GO:0030878',
  'GO:0030878',
  'GO:0038194',
  'GO:0039003',
  'GO:0042472',
  'GO:004

### Pre computando todos os termos GO

In [3]:
all_go_terms = []
for i in list(affymetrix_GO_terms_dict.values()):
    all_go_terms.extend(i)
all_go_terms = list(set(all_go_terms))

In [6]:
with open('./Data/go_terms_14520_U133A.pkl', 'wb') as arquivo:
    # Serializar e salvar a lista no arquivo
    pickle.dump(all_go_terms, arquivo)

In [9]:
array = np.empty([len(all_go_terms),len(all_go_terms)], dtype='float16')

In [10]:
for i in range(200):
    array[i*10, 300] = 10
start = time.time()
array[100,300]
print(time.time() - start)

9.632110595703125e-05


In [11]:
pool = multiprocessing.Pool(48)
def f(gen1,gen2):
    return Similarity_of_Two_GOTerms(gen1, gen2, go, 'GOGO')

In [12]:
for i in range(len(all_go_terms)):
    gen1 = all_go_terms[i]
    array[i] = pool.starmap(f, zip(repeat(gen1), all_go_terms))
    print(print(array[i]))

[1.      0.2617  0.1318  ... 0.02263 0.0869  0.1842 ]
None
[0.2617  1.      0.138   ... 0.02303 0.1105  0.2019 ]
None
[0.1318  0.138   1.      ... 0.02138 0.0451  0.1792 ]
None
[0.0786 0.079  0.0767 ... 0.3118 0.0799 0.078 ]
None
[0.0373  0.04126 0.02742 ... 0.04257 0.2129  0.03146]
None
[0.0756  0.09204 0.04272 ... 0.04953 0.1484  0.05948]
None
[0.1268  0.1678  0.0974  ... 0.0922  0.04257 0.1398 ]
None
[0.2201  0.561   0.1941  ... 0.02312 0.07733 0.1805 ]
None
[0.05637 0.06018 0.04398 ... 0.1137  0.07214 0.05115]
None
[0.1904  0.296   0.11554 ... 0.02391 0.0973  0.2142 ]
None
[0.03525 0.03726 0.02914 ... 0.04507 0.2334  0.03195]
None
[0.07196 0.07666 0.05585 ... 0.1346  0.0894  0.06586]
None
[0.3767  0.4614  0.1794  ... 0.02454 0.1696  0.2847 ]
None
[0.05725 0.06064 0.04587 ... 0.2112  0.07086 0.0526 ]
None
[0.3909  0.2389  0.1537  ... 0.02373 0.11115 0.1593 ]
None
[0.2335 0.2659 0.3474 ... 0.023  0.0974 0.2852]
None
[0.07574 0.0951  0.04065 ... 0.0479  0.1669  0.05774]
None
[0.05875 

In [22]:
go_terms_index = dict(zip(all_go_terms, range(len(all_go_terms))))
array[go_terms_index['GO:2000392'], go_terms_index['GO:2000392']]

1.0

### Salvando os arquivos

In [28]:
np.save('go_terms_GOGO.npy', array)
with open('dict_go_terms_index.pkl', 'wb') as file:
    pickle.dump(go_terms_index, file)

In [13]:
for i in range(len(colunas)):
    for j in range(len(colunas)):
        gen1 = affymetrix_GO_terms_dict[colunas[i]]
        gen2 = affymetrix_GO_terms_dict[colunas[j]]
        if gen1 != [] and gen2 != []:
            print(Similarity_of_Set_of_GOTerms(gen1, gen2, 'GOGO'))

1.0
0.269
0.275
0.411
0.311
0.35
0.352
0.338


KeyboardInterrupt: 

In [29]:
len(set(go_terms_sets))

10315

### Pre computando affymetrix agora

In [3]:
go_terms_sets = [affymetrix_GO_terms_dict[k] for k in colunas]

go_terms_sets_unique = []
for i in list(set(tuple(x) for x in go_terms_sets)):
    go_terms_sets_unique.append(list(i))

In [4]:
matrix_similarity = np.load('go_terms_GOGO.npy')
with open('dict_go_terms_index.pkl', 'rb') as file:
    go_labels = pickle.load(file)

In [5]:
def init_worker(matrix_similarity, go_labels):
    global matrix_similarity_worker
    matrix_similarity_worker = np.copy(matrix_similarity)
    global go_labels_worker
    go_labels_worker = go_labels.copy()
    global colunas_worker
    colunas_worker = colunas.copy()

def f_gen(gen1,gen2):
    if len(gen1) != 0 and len(gen2) != 0:
        return Similarity_of_Set_of_GOTerms(gen1, gen2, 'pre_computed_GOGO', matrix_similarity = matrix_similarity_worker, go_labels = go_labels_worker)
    else:
        return 0
        
pool = multiprocessing.Pool(48, init_worker, [matrix_similarity, go_labels])

In [6]:
matrix_go_terms_sets = np.empty([len(go_terms_sets_unique),len(go_terms_sets_unique)], dtype='float16')
for i in trange(len(go_terms_sets_unique)):
    gen1 = go_terms_sets_unique[i]
    matrix_go_terms_sets[i] = pool.starmap(f_gen, zip(repeat(gen1), go_terms_sets_unique))
    #print(matrix_affymetrix[i])

  0%|          | 0/10315 [00:00<?, ?it/s]

In [14]:
list(zip(repeat(gen1), [affymetrix_GO_terms_dict[k] for k in colunas]))

[(['GO:0001558',
   'GO:0001952',
   'GO:0006468',
   'GO:0007155',
   'GO:0007169',
   'GO:0007565',
   'GO:0007566',
   'GO:0007595',
   'GO:0008285',
   'GO:0010715',
   'GO:0014909',
   'GO:0016310',
   'GO:0018108',
   'GO:0030198',
   'GO:0038063',
   'GO:0038063',
   'GO:0038083',
   'GO:0043583',
   'GO:0044319',
   'GO:0046777',
   'GO:0060444',
   'GO:0060749',
   'GO:0061302'],
  ['GO:0001558',
   'GO:0001952',
   'GO:0006468',
   'GO:0007155',
   'GO:0007169',
   'GO:0007565',
   'GO:0007566',
   'GO:0007595',
   'GO:0008285',
   'GO:0010715',
   'GO:0014909',
   'GO:0016310',
   'GO:0018108',
   'GO:0030198',
   'GO:0038063',
   'GO:0038063',
   'GO:0038083',
   'GO:0043583',
   'GO:0044319',
   'GO:0046777',
   'GO:0060444',
   'GO:0060749',
   'GO:0061302']),
 (['GO:0001558',
   'GO:0001952',
   'GO:0006468',
   'GO:0007155',
   'GO:0007169',
   'GO:0007565',
   'GO:0007566',
   'GO:0007595',
   'GO:0008285',
   'GO:0010715',
   'GO:0014909',
   'GO:0016310',
   'GO:0018

In [50]:
'''
Sim, dá certo, testei, confia
go_terms_sets = []
for i in list((affymetrix_GO_terms_dict.values())):
    go_terms_sets.append(str(i))
go_terms_sets = list(set(go_terms_sets))
'''

In [7]:
matrix_go_terms_sets

array([[1.   , 0.136, 0.531, ..., 0.14 , 0.294, 0.347],
       [0.136, 1.   , 0.15 , ..., 0.06 , 0.21 , 0.148],
       [0.531, 0.15 , 1.   , ..., 0.1  , 0.194, 0.273],
       ...,
       [0.14 , 0.06 , 0.1  , ..., 1.   , 0.188, 0.151],
       [0.294, 0.21 , 0.194, ..., 0.188, 1.   , 0.407],
       [0.347, 0.148, 0.273, ..., 0.151, 0.407, 1.   ]], dtype=float16)

In [17]:
go_terms_sets_labels = dict(zip([tuple(k) for k in go_terms_sets_unique], range(len(go_terms_sets_unique))))

In [36]:
matrix_go_terms_sets[go_terms_sets_labels[tuple(affymetrix_GO_terms_dict[colunas[12000]])],[go_terms_sets_labels[tuple(affymetrix_GO_terms_dict[colunas[13000]])]]]

array([0.388], dtype=float16)

In [45]:
def get_Similarity_of_Set_of_GOTerms(gen1, gen2, matrix_similarity, labels):
    return matrix_similarity[labels[tuple(gen1)]][[labels[tuple(gen2)]]][0]

In [157]:
gen1 = affymetrix_GO_terms_dict[colunas[13000]]
gen2 = affymetrix_GO_terms_dict[colunas[12000]]
start = time.time()
get_Similarity_of_Set_of_GOTerms(gen1, gen2, matrix_go_terms_sets, go_terms_sets_labels)
print(time.time() - start)

start = time.time()
Similarity_of_Set_of_GOTerms(gen1, gen2, 'GOGO')
print(time.time() - start)


0.00014829635620117188
0.13811230659484863


In [158]:
gen1 = affymetrix_GO_terms_dict[colunas[np.random.randint(0,22000)]]
gen2 = affymetrix_GO_terms_dict[colunas[np.random.randint(0,22000)]]
gen3 = affymetrix_GO_terms_dict[colunas[np.random.randint(0,22000)]]
gen4 = affymetrix_GO_terms_dict[colunas[np.random.randint(0,22000)]]
gen5 = affymetrix_GO_terms_dict[colunas[np.random.randint(0,22000)]]


In [159]:
genxt = gen1.copy()
genxt.extend(gen2)
genxt.extend(gen3)
genxt.extend(gen4)

In [160]:
Similarity_of_Set_of_GOTerms(genxt, gen5, 'GOGO')

0.223

In [161]:
(get_Similarity_of_Set_of_GOTerms(gen1, gen5, matrix_go_terms_sets, go_terms_sets_labels) + get_Similarity_of_Set_of_GOTerms(gen2, gen5, matrix_go_terms_sets, go_terms_sets_labels) + get_Similarity_of_Set_of_GOTerms(gen3, gen5, matrix_go_terms_sets, go_terms_sets_labels) + get_Similarity_of_Set_of_GOTerms(gen4, gen5, matrix_go_terms_sets, go_terms_sets_labels))/4

0.1116943359375

In [162]:
print('1', gen1)
print('1', gen2)
print('1', gen3)
print('1', gen4)
print('1', gen5)

1 ['GO:0006977', 'GO:0007050', 'GO:0008630']
1 ['GO:0006417', 'GO:0017148', 'GO:0045727']
1 ['GO:0005975', 'GO:0006068', 'GO:0008152', 'GO:0055114']
1 []
1 ['GO:0006600', 'GO:0006810', 'GO:0006811', 'GO:0006814', 'GO:0006836', 'GO:0006936', 'GO:0015871', 'GO:0015881', 'GO:0034641', 'GO:0044281', 'GO:0055085', 'GO:1902598', 'GO:1902598', 'GO:1902598']


In [91]:
go_terms_sets_labels

{('GO:0001558',
  'GO:0006281',
  'GO:0006325',
  'GO:0006351',
  'GO:0006355',
  'GO:0006974',
  'GO:0016568',
  'GO:0040008',
  'GO:0045944',
  'GO:0051155'): 0,
 ('GO:0038032',): 1,
 ('GO:0006351',
  'GO:0006355',
  'GO:0006915',
  'GO:0043065',
  'GO:0043620',
  'GO:0045892',
  'GO:0045892',
  'GO:2000144',
  'GO:2001022',
  'GO:2001244'): 2,
 ('GO:0002376',
  'GO:0019722',
  'GO:0035556',
  'GO:0038095',
  'GO:0042113',
  'GO:0042113',
  'GO:0043303',
  'GO:0045087',
  'GO:0050853'): 3,
 ('GO:0001974',
  'GO:0001975',
  'GO:0001987',
  'GO:0001996',
  'GO:0001997',
  'GO:0006937',
  'GO:0007165',
  'GO:0007186',
  'GO:0007188',
  'GO:0007267',
  'GO:0007275',
  'GO:0007512',
  'GO:0007626',
  'GO:0008283',
  'GO:0008542',
  'GO:0016049',
  'GO:0019229',
  'GO:0035265',
  'GO:0035556',
  'GO:0042593',
  'GO:0043278',
  'GO:0043410',
  'GO:0045818',
  'GO:0045819',
  'GO:0048148',
  'GO:0055117',
  'GO:0071875',
  'GO:0071875'): 4,
 ('GO:0000122',
  'GO:0001501',
  'GO:0002062',
  '

### Salvando os arquivos

In [164]:
np.save('Data/matrix_go_terms_sets.npy', matrix_go_terms_sets)
with open('Data/go_terms_sets_labels.pkl', 'wb') as file:
    pickle.dump(go_terms_sets_labels, file)